In [1]:
from __future__ import division
def feature_eng(y, fs, vis=False):
    
    import librosa
    import librosa.display
    import numpy as np
    import maad
    from soundsig.sound import BioSound
    import matplotlib.pyplot as plt
    import pandas as pd
    import warnings
    warnings.simplefilter('ignore')
    
    # spectrum
    ps = np.abs(np.fft.fft(y))**2
    time_step = 1/fs
    freqs = np.fft.fftfreq(y.size, time_step)
    ps = ps[0:int((len(ps)/2)-1)] # take out the negative freq
    freqs = freqs[0:int((len(freqs)/2)-1)]

    # melspectrogram
    S = librosa.feature.melspectrogram(y=y, sr=fs)
    S_dB = librosa.power_to_db(S, ref=np.max)
    
    # alpha indices
    Sxx_power,tn,fn,ext = maad.sound.spectrogram (y, fs, mode='psd')
    df_temporal_indices = maad.features.all_temporal_alpha_indices(y, fs)
    df_spectral_indices, _ = maad.features.all_spectral_alpha_indices(Sxx_power,tn,fn, extent=ext)
    df_indices = pd.concat([df_temporal_indices,df_spectral_indices], axis=1)
    
    # soundsig for modulation power spectrum
    
    myBioSound = BioSound(soundWave=y, fs=fs)
    myBioSound.mpsCalc(window=1, Norm = True)
    
    # reduce the dimension of MPS to one quadrant
    len1 = int(len(myBioSound.wf-1)/2)
    len2 = int(len(myBioSound.wt-1)/2)
    quad1 = myBioSound.mps[len1:,len2:]
    quad2 = np.fliplr(myBioSound.mps[len1:,:len2+1])
    mps = (quad1+quad2)/2
    wf = myBioSound.wf[len1:]
    wt = myBioSound.wt[len2:]
    
    if vis==True:
        plt.figure()
        plt.plot(np.arange(0,y.size/fs,1/fs), y)
        plt.show()
        
        fig, ax = plt.subplots()
        img = librosa.display.specshow(S_dB, x_axis='time',
                                 y_axis='mel', sr=fs,
                                 fmax=fs/2, ax=ax)
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set(title='Mel-frequency spectrogram')
        
        DBNOISE=100
        plt.figure()
        plt.clf()
        cmap = plt.get_cmap('jet')
        ex = (myBioSound.wt.min(), myBioSound.wt.max(), myBioSound.wf.min()*1e3, myBioSound.wf.max()*1e3)
        logMPS = 10.0*np.log10(myBioSound.mps)
        maxMPS = logMPS.max()
        minMPS = maxMPS-DBNOISE
        logMPS[logMPS < minMPS] = minMPS
        plt.imshow(logMPS, interpolation='nearest', aspect='auto', origin='lower', cmap=cmap, extent=ex)
        plt.ylabel('Spectral Frequency (Cycles/KHz)')
        plt.xlabel('Temporal Frequency (Hz)')
        plt.colorbar()
        plt.ylim((0,myBioSound.wf.max()*1e3))
        plt.title('Modulation Power Spectrum')
        plt.show()
        
    
    return ps, freqs, S_dB, df_indices, mps, wt, wf
    

In [2]:
def preproc(file, fs=16000, vis=False):
    import librosa
    import numpy as np
    import matplotlib.pyplot as plt
    import noisereduce as nr
    
    raw_y, fs = librosa.load(file, sr=fs, duration=10, mono = True)
    y_mono_rs = raw_y - np.mean(raw_y) # remove DC
    rms = np.sqrt(np.mean(y_mono_rs**2)) # get rms
    y = y_mono_rs/(rms/0.1) # normalize the rms to 0.1
    
    fg_y = nr.reduce_noise(y=y, sr=fs)
    bg_y = y - fg_y
    
    
    if vis == True: print('++++++++++++++++++++++++ raw ++++++++++++++++++++++++')
    ps, freqs, S_dB, df_indices, mps, wt, wf = feature_eng(y, fs, vis)
    if vis == True: print('++++++++++++++++++++ foreground ++++++++++++++++++++')
    ps_fg, freqs_fg, S_dB_fg, df_indices_fg, mps_fg, wt_fg, wf_fg = feature_eng(fg_y, fs, vis)
    if vis == True: print('++++++++++++++++++++ background ++++++++++++++++++++')
    ps_bg, freqs_fg, S_dB_bg, df_indices_bg, mps_bg, wt_bg, wf_bg = feature_eng(bg_y, fs, vis)
    
    output = {'fs': fs, 'y': y, 'fg_y': fg_y, 'bg_y': bg_y,
             'df_indices': df_indices, 'mps': mps, 'wt': wt, 'wf': wf, 
             'df_indices_fg': df_indices_fg, 'mps_fg': mps_fg,
             'df_indices_bg': df_indices_bg, 'mps_bg': mps_bg}
    
    return output
    

In [3]:
# scan data directories
import glob
import pandas as pd

nature_file_list = []
nature_file_list += glob.glob('../data/raw/AmbisonicSoundLibrary/nature/*')
nature_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/raw/youtube/NatureSoundscapes/*.m4a')
nature_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_nature/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/non_urban/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/urban_0_25/*')

nature_file_list.remove('../data/raw/youtube/NatureSoundscapes/xRrqcK46roE_19125.m4a') # this file is blank

city_file_list = []
city_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_city/*')
city_file_list += glob.glob('../data/raw/S2L_LULC/urban_26_100/*')

In [4]:
# as there are too many files in SONYC, so only a few are sampled
from random import seed, sample
len_diff = len(nature_file_list)-len(city_file_list) # how many SONYC files need to add to make the dataset balanced

seed(23)
SONYC_file_list = sample(glob.glob('../data/raw/SONYC/**/*.wav', recursive=True),len_diff)
city_file_list += SONYC_file_list

pd.DataFrame({'nature_file_list': nature_file_list}).to_csv('../data/raw/nature_file_list.csv')
pd.DataFrame({'city_file_list': city_file_list}).to_csv('../data/raw/city_file_list.csv')

In [5]:
print('There are '+str(len(nature_file_list))+' files per category.')

There are 962 files per category.


In [6]:
# testing cell
# preproc(city_file_list[0])

In [7]:
import time
import pickle
import os

from joblib import Parallel, delayed

def run_preproc(file_name):
    save_file_name = '../data/interim/'+file_name[12:-4]+'.pkl'
    if not os.path.isfile(save_file_name): # run the script only if the file does not exist
        if not os.path.exists(save_file_name.rsplit('/', 1)[0]): # create the folder if the folder does not exist
            os.makedirs(save_file_name.rsplit('/', 1)[0]) # extract the folder of the file path
            

        start_time = time.time()

        output = preproc(file_name, vis = False)
        f = open(save_file_name,'wb') # create a binary pickle file 
        pickle.dump(output,f)
        f.close()

        print("--- %s seconds ---" % (time.time() - start_time))
        

# run the process in parallel
Parallel(n_jobs=-1)(delayed(run_preproc)(file_name) for file_name in nature_file_list+city_file_list)

--- 18.049998998641968 seconds ---
--- 10.594537019729614 seconds ---
--- 8.812788963317871 seconds ---
--- 9.431322813034058 seconds ---


/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


--- 18.19015097618103 seconds ---
--- 10.52818489074707 seconds ---
--- 8.954497575759888 seconds ---
--- 9.64345407485962 seconds ---
--- 18.209975957870483 seconds ---
--- 10.50726580619812 seconds ---
--- 9.016366004943848 seconds ---
--- 9.622343063354492 seconds ---
--- 18.28684902191162 seconds ---
--- 10.611193895339966 seconds ---
--- 8.872929096221924 seconds ---
--- 9.606220006942749 seconds ---
--- 18.250075101852417 seconds ---
--- 10.541741609573364 seconds ---
--- 8.912078857421875 seconds ---
--- 9.639028072357178 seconds ---
--- 18.19934630393982 seconds ---
--- 10.562231063842773 seconds ---
--- 8.913066864013672 seconds ---
--- 9.681729078292847 seconds ---
--- 8.794387817382812 seconds ---
--- 9.090636968612671 seconds ---
--- 8.930835008621216 seconds ---
--- 12.79351019859314 seconds ---
--- 9.540361881256104 seconds ---
--- 11.470783948898315 seconds ---
--- 16.150476932525635 seconds ---
--- 11.096079111099243 seconds ---
--- 9.548954963684082 seconds ---
--- 9.1

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [8]:
# for file_name in nature_file_list+city_file_list:
#     print(file_name)
#     run_preproc(file_name)

--- 14.985634803771973 seconds ---
--- 8.266017198562622 seconds ---
--- 12.334496021270752 seconds ---
--- 10.095956087112427 seconds ---
--- 9.937620162963867 seconds ---
--- 15.707894086837769 seconds ---
--- 12.660273313522339 seconds ---
--- 9.867726802825928 seconds ---
--- 9.273952007293701 seconds ---
--- 9.048890829086304 seconds ---
--- 11.251073837280273 seconds ---
--- 10.518306970596313 seconds ---
--- 9.170018911361694 seconds ---
--- 9.154603958129883 seconds ---
--- 9.54865312576294 seconds ---
--- 9.648802995681763 seconds ---
--- 9.624490976333618 seconds ---
--- 10.39084005355835 seconds ---
--- 9.68632173538208 seconds ---
--- 9.233370304107666 seconds ---
--- 9.577141046524048 seconds ---
--- 9.690654039382935 seconds ---
--- 9.689582824707031 seconds ---
--- 9.515312194824219 seconds ---
--- 10.249928951263428 seconds ---
--- 10.383188962936401 seconds ---
--- 9.551735162734985 seconds ---
--- 9.301352977752686 seconds ---
--- 10.048582792282104 seconds ---
--- 9.

--- 14.962491035461426 seconds ---
--- 9.073173761367798 seconds ---
--- 12.905310153961182 seconds ---
--- 9.513062238693237 seconds ---
--- 11.618775129318237 seconds ---
--- 16.73479413986206 seconds ---
--- 10.378332138061523 seconds ---
--- 9.572843074798584 seconds ---
--- 8.934393882751465 seconds ---
--- 9.726507902145386 seconds ---
--- 11.005483865737915 seconds ---
--- 10.128966093063354 seconds ---
--- 9.060626029968262 seconds ---
--- 9.152029037475586 seconds ---
--- 9.486110925674438 seconds ---
--- 9.1598379611969 seconds ---
--- 9.568009853363037 seconds ---
--- 10.17565107345581 seconds ---
--- 9.293484687805176 seconds ---
--- 9.226340770721436 seconds ---
--- 9.277162790298462 seconds ---
--- 9.766181945800781 seconds ---
--- 9.713070154190063 seconds ---
--- 9.840855121612549 seconds ---
--- 11.12100076675415 seconds ---
--- 10.239524126052856 seconds ---
--- 9.640709161758423 seconds ---
--- 9.475237131118774 seconds ---
--- 10.231499910354614 seconds ---
--- 9.25